<a href="https://colab.research.google.com/github/RrobsonFreitas/Desafio_Alura/blob/main/Desafio_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Atualiza a biblioteca google-generativeai para a versão mais recente
!pip install -q -U google-generativeai

In [ ]:
# Instalando a biblioteca que permite ler e manipular arquivos PDF.
!pip install pypdf2

In [19]:
# Importando as bibliotecas (pathlib, hashlib, google.generativeai, PyPDF2)
from pathlib import Path
import hashlib
import google.generativeai as genai
import PyPDF2

In [20]:
# Configura a API do Google Generative AI com uma chave secreta armazenada no Google Colab.
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [22]:
# Configurando os parâmetros de geração de texto
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
 }

In [23]:
# Configurando as definições de segurança para bloquear conteúdo prejudicial, como assédio, discurso de ódio, conteúdo sexualmente explícito e conteúdo perigoso.
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]


In [24]:
# Inicializando o modelo generativo chamado "gemini-1.5-pro-latest" com as configurações de geração de texto e segurança especificadas anteriormente.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [25]:
# Fazer o upload do arquivo no ambiente Google Colab.
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [26]:
#  Definir a função extract_pdf_pages que extrai o texto de cada página de um arquivo PDF e retorna uma lista contendo o texto extraído.
def extract_pdf_pages(pathname: str) -> list[str]:
  parts = [f"--- START OF PDF ${pathname} ---"]
  with open(pathname, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    num_pages = len(pdf_reader.pages)

    for page_num in range(num_pages):
      page = pdf_reader.pages[page_num]
      page_content = page.extract_text()
      parts.append(f"--- PAGE {page_num} ---")
      parts.append(page_content)

  return parts

In [27]:
# Função para verificar se a resposta está dentro do escopo do assunto desejado
def esta_no_escopo(resposta):
  palavras_chave = ["porquinho", "3 porquinhos", "livro", "lobo", "casa", "história"]
  return any(palavra in resposta.lower() for palavra in palavras_chave)

In [28]:
# Função responder_pergunta que recebe uma pergunta e um contexto; e verifica se a resposta está dentro do escopo da história.
# Se a resposta estiver fora do escopo, a função retorna uma mensagem indicando sua especialidade.
# Contêm instruções para responder somente sobre o arquivo importado para a base de conhecimento.

def responder_pergunta(pergunta, contexto):
  instrucoes = """
Seu objetivo é ajudar as pessoas a entender a história dos 3 prquinhos.
Responda somente assuntos relacionados ao conteúdo do documento anexado na sua base de conhecimento.
Caso algum assunto fuja do conteúdo, responda amigavelmente que sua principal função é tratar apenas sobre a história dos 3 porquinhos.
Tente responder de forma resumida apenas o que for perguntado, só acrescente mais informações se for solicitado.
"""
  prompt = f"{instrucoes}\nContexto: {contexto}\nPergunta: {pergunta}\nResposta:"
  resposta = model.generate_content(prompt).text
  if not esta_no_escopo(resposta):
    resposta = "Desculpe, minha especialidade é a história dos 3 porquinhos. Só posso responder perguntas relacionadas a esse tema com base no meu conhecimento."
  return resposta

In [32]:
# Extrair o texto de todas as páginas do arquivo e armazenar o texto completo em uma única string chamada texto_completo.
pdf_path = "3porquinhos.pdf"
texto_do_livro = extract_pdf_pages(pdf_path)
texto_completo = "\n".join(texto_do_livro)

In [33]:
# Comandos que permite ao usuário fazer perguntas sobre o livro dos 3 porquinhos.
# O chatbot gera respostas usando a função responder_pergunta e o loop continua até que o usuário digite "sair".
print("Olá! Sou um chatbot que pode responder perguntas sobre o livro dos 3 porquinhos.")
while True:
  pergunta = input("Digite sua pergunta (ou 'sair' para encerrar): ")
  if pergunta.lower() == "sair":
    break
  resposta = responder_pergunta(pergunta, texto_completo)
  print(resposta)

Olá! Sou um chatbot que pode responder perguntas sobre o livro dos 3 porquinhos.
Digite sua pergunta (ou 'sair' para encerrar): sair
